### Imports

In [1]:
from pylab import *
%matplotlib inline
rcParams['figure.figsize'] = (15.0, 30.0)

* Prepare paths

In [2]:
import sys
sys.path.append('..')

### General data structures

* Preload data

In [3]:
from dataset import get_positive_patches, get_negative_patches, get_labels, load_image
from common import dataset, utils
import numpy as np
import pandas as pd

labels_data, data = dataset.labels_data, dataset.data


## This one contains separatedly cars, vans...
#vehicle_types = set(dataset.map_category.values())
#vehicle_types.add('Empty')

map_category = {'A':'Vehicle', 'B':'Vehicle', 'C':'Vehicle', 'D':'Vehicle', 'E':'Vehicle', 'F':'Vehicle', 'G':'Vehicle', 'H':'Vehicle', 'I':'Vehicle'}
labels_data = pd.read_csv('../data/trainingObservations.csv')
data = get_labels(labels_data, map_category)
vehicle_types = set(map_category.values())

* Split train and test

In [4]:
#test_images = 60
#cases = list(data.keys())
#np.random.shuffle(cases)
#train_cases, test_cases = cases[test_images:], cases[:test_images]
#np.save('../data/patch_images/train_cases.npy', train_cases)
#np.save('../data/patch_images/test_cases.npy', test_cases)
train_cases = np.load('../data/patch_images/train_cases.npy')
test_cases  = np.load('../data/patch_images/test_cases.npy')

In [5]:
vehicle_types.add('Empty')

In [6]:
image_size_nn = 48

### Train a resnet50

In [185]:
import os
import pandas as pd
annotations_path = '../data/patch_images/inferred_annotations'
predicted_files = os.listdir(annotations_path)
min_thrs = 20

In [211]:
def define_solution(row):
    if ~np.isnan(row['xpred']) and ~np.isnan(row['ygnd']):
        return 'TP'
    elif ~np.isnan(row['xpred']) and np.isnan(row['ygnd']):
        return 'FP'
    elif np.isnan(row['xpred']) and ~np.isnan(row['ygnd']):
        return 'FN'

def plot_df(filename, df):
    imshow(dataset.load_image(filename))
    plot(df[df['type'] == 'TP']['xpred'].values, df[df['type'] == 'TP']['ypred'].values, 'og')
    plot(df[df['type'] == 'FP']['xpred'].values, df[df['type'] == 'FP']['ypred'].values, 'or')
    plot(df[df['type'] == 'FN']['xgnd'].values, df[df['type'] == 'FN']['ygnd'].values, 'oy')


def run_eval(filename):
    try:
        predloc = pd.read_csv("%s/%s" % (annotations_path,filename), names=['x','y'])
        predloc['point'] = None
    except:
        predloc = pd.DataFrame(columns = ['x','y'])
        
    groundloc = pd.DataFrame(data=data[filename]['Vehicle'], columns=['y','x'])
    groundloc['point'] = None        

    if groundloc.shape[0] > 0:
        for i_pred in predloc.index:

            aux = (groundloc[['x','y']] - predloc.ix[i_pred][['x','y']]).abs().max(axis=1)
            min_index = aux.argmin()
            min_value = aux.min()

            predloc.loc[i_pred, 'point'] = i_pred
            if min_value <= min_thrs:
                groundloc.loc[min_index, 'point'] = i_pred
    
    df = pd.merge(predloc, groundloc, on = 'point', how = 'outer', suffixes=['pred','gnd'])
    if df.index.shape[0] == 0:
        return None, None

    df['type'] = df.apply(define_solution, axis = 1)
    df['filename'] = filename
    res_df = df[['point','type','filename']].fillna(0).groupby(['filename','type']).count().unstack('filename').T
    return df, res_df

dfs, res_dfs = [], []
for filename in predicted_files:
    df, res_df = run_eval(filename)
    dfs.append(df)
    res_dfs.append(res_df)

res_dfs = pd.concat(res_dfs).ix['point']
dfs = pd.concat(dfs)

In [212]:
a = res_dfs.sum(axis=0)
print(a['TP'] / (a['TP']+a['FP']+a['FN']), res_dfs.index.shape)
#res_dfs

0.484555984556 (21,)


In [188]:
#file = 'TQ2379_6_0'
#plot_df(file, dfs[dfs.filename == file])

In [215]:
ls ../data/patch_images/inferred_annotations/ | wc -l

26
